# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint as prettyprint

# Import API key
from api_keys import g_key
from api_keys import weather_api_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv('../output_data/cities.csv')
cities_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Tanout,14.97,8.89,94.03,30,45,16.67,NE,1595269118
1,Bluff,-46.60,168.33,39.00,83,100,3.00,NZ,1595269118
2,Arraial do Cabo,-22.97,-42.03,80.31,69,0,13.87,BR,1595269008
3,Mataura,-46.19,168.86,39.00,83,100,3.00,NZ,1595269004
4,Dikson,73.51,80.55,42.28,93,65,14.45,RU,1595269010


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key)



In [11]:
locations = cities_df[["Lat", "Lng"]]
weights = cities_df["Humidity"]
center = (10,0)


In [13]:


fig = gmaps.figure(center = center, zoom_level = 2.1)
fig.add_layer(gmaps.heatmap_layer(locations, weights = weights, dissipating=False, max_intensity=100,
                                 point_radius = 5))
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
per =cities_df["Max Temp"] > 80
perf = cities_df.drop(cities_df.index[per])
perfec = perf["Max Temp"] < 70
perfect = perf.drop(perf.index[perfec])
perfect_w = perfect["Cloudiness"] = 0
perfect_weth = perfect.drop(perfect.index[perfect_w])
perfect_wethr = perfect_weth["Wind Speed"] > 10
perfect_weathe = perfect_weth.drop(perfect_weth.index[perfect_wethr])
perfect_s_weather = perfect_weathe["Lat"] < 0
perfect_weather = perfect_weathe.drop(perfect_weathe.index[perfect_s_weather])
perfect_weather


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,Laguna,38.42,-121.42,72.81,56,0,9.17,US,1595269124
7,Nikolskoye,59.70,30.79,70.84,64,0,4.47,RU,1595269125
14,Qui Nhon,13.77,109.23,79.36,85,0,4.00,VN,1595269040
15,Carbonear,47.73,-53.21,78.33,100,0,5.01,CA,1595268903
16,Murmashi,68.82,32.81,71.60,64,0,4.47,RU,1595269129
...,...,...,...,...,...,...,...,...,...
567,Erzurum Province,40.00,41.50,71.60,35,0,4.70,TR,1595269051
571,Lanxi,29.22,119.48,77.74,90,0,1.28,CN,1595269261
572,Cotacachi,0.30,-78.27,73.40,31,0,8.05,EC,1595269261
576,Dillingen,49.35,6.73,75.43,50,0,9.17,DE,1595269262


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
hotel_df = perfect_weather
hotel_df['Hotel Name'] = ""


In [19]:
# geocoordinates
target_search = "hotel"
target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key,

}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
 

    # get coordinates  from df
    #target_coordinates = row["Lat"], row["Lng"]
    #print(target_coordinates)
    
    # add keyword to params dict
    params['location'] = f"{lat},{lng}"

    # assemble url and make API request
    #print(f"Retrieving Results for Index {index}: {row[0]}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    try:
        #print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']

    except (KeyError, IndexError):
        print(f'{row[0]} has no hotels')
 
    

Murmashi has no hotels
Partyzany has no hotels
Oruzgan has no hotels
Lere has no hotels
Petropavlivka has no hotels
Ostrovnoy has no hotels
Kanelovskaya has no hotels
Palkino has no hotels
Nalvo has no hotels
Dhilwan has no hotels
Yuli has no hotels
Bondo has no hotels
Karatau has no hotels
Yaan has no hotels
Marathon has no hotels
Rangāpāra has no hotels
Erzurum Province has no hotels
Lumphat has no hotels


In [20]:

hotel_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
6,Laguna,38.42,-121.42,72.81,56,0,9.17,US,1595269124,Hilton Garden Inn Sacramento Elk Grove
7,Nikolskoye,59.70,30.79,70.84,64,0,4.47,RU,1595269125,Tourist House - Sablino
14,Qui Nhon,13.77,109.23,79.36,85,0,4.00,VN,1595269040,Mento Hotel
15,Carbonear,47.73,-53.21,78.33,100,0,5.01,CA,1595268903,Hotel Harbour Grace
16,Murmashi,68.82,32.81,71.60,64,0,4.47,RU,1595269129,
...,...,...,...,...,...,...,...,...,...,...
567,Erzurum Province,40.00,41.50,71.60,35,0,4.70,TR,1595269051,
571,Lanxi,29.22,119.48,77.74,90,0,1.28,CN,1595269261,Shangri-La Hotel
572,Cotacachi,0.30,-78.27,73.40,31,0,8.05,EC,1595269261,La Mirage Garden Hotel And Spa
576,Dillingen,49.35,6.73,75.43,50,0,9.17,DE,1595269262,LA MAISON hotel Saarlouis


In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
weights = hotel_df["Humidity"]
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

In [22]:
# Add marker layer ontop of heat map
# Display figure
fig = gmaps.figure(center = center, zoom_level = 2.1)
fig.add_layer(gmaps.heatmap_layer(locations, weights = weights, dissipating=False, max_intensity=100,
                                 point_radius = 5))
fig.add_layer(marker_layer)
fig


Figure(layout=FigureLayout(height='420px'))